In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import scipy.stats as stats
from scikit_posthocs import posthoc_dunn

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## 87 POIs

In [21]:
def result_to_xlsx(result,n,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        for j in range(len(result[i])):
            temp = pd.DataFrame(result[i][j],columns=["fitness","run_time"])
            temp['n'] = n[j]
            temp['method'] = i
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [22]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "aco_vrp": [],
    "aco_tsp": [],
    "haco_vrp": [],
    "haco_tsp": [],
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(5,(5*30)+1,5))
    random_node = {
        "aco_vrp": [],
        "aco_tsp": [],
        "haco_vrp": [],
        "haco_tsp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_vrp"].append([results_aco_vrp[key] for key in ["fitness","run_time"]])

        aco_tsp = ACO_TSP(random_state=100)
        results_aco_tsp = get_analysis(aco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_tsp"].append([results_aco_tsp[key] for key in ["fitness","run_time"]])

        haco_vrp = HACO_VRP(random_state=100)
        results_haco_vrp = get_analysis(haco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_vrp"].append([results_haco_vrp[key] for key in ["fitness","run_time"]])

        haco_tsp = HACO_TSP(random_state=100)
        results_haco_tsp = get_analysis(haco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_tsp"].append([results_haco_tsp[key] for key in ["fitness","run_time"]])
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
n =  10
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
rando

random_state :  150
n =  75
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
n =  80
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
rand

In [23]:
run_time_analysis

{'aco_vrp': [[[0.6983024691358025, 0.2414865493774414],
   [0.7066049382716048, 0.24062275886535645],
   [0.7013374485596708, 0.23910856246948242],
   [0.7109053497942391, 0.3020339012145996],
   [0.7120884773662551, 0.21251845359802246],
   [0.8868004115226338, 0.21419882774353027],
   [0.6891460905349795, 0.21042585372924805],
   [0.6708847736625518, 0.27855682373046875],
   [0.710133744855967, 0.281663179397583],
   [0.7318415637860084, 0.25290703773498535],
   [0.7183641975308642, 0.22420573234558105],
   [0.727366255144033, 0.21828866004943848],
   [0.6858024691358028, 0.28426098823547363],
   [0.6817386831275719, 0.2161872386932373],
   [0.7270061728395064, 0.2393052577972412],
   [0.6945781893004117, 0.22478580474853516],
   [0.675565843621399, 0.21371960639953613],
   [0.683590534979424, 0.2655799388885498],
   [0.6993312757201645, 0.27854299545288086],
   [0.6887860082304527, 0.32664942741394043],
   [0.6979526748971194, 0.21361470222473145],
   [0.6787037037037035, 0.22607636

In [24]:
result_to_xlsx(run_time_analysis,n,random_ls,"3_various_numbers_pois_all_1_haco.xlsx")

In [40]:
list(range(155,(5*50)+1,5))

20

In [41]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "aco_vrp": [],
    "aco_tsp": [],
    "haco_vrp": [],
    "haco_tsp": [],
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(155,(5*50)+1,5))
    random_node = {
        "aco_vrp": [],
        "aco_tsp": [],
        "haco_vrp": [],
        "haco_tsp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_vrp"].append([results_aco_vrp[key] for key in ["fitness","run_time"]])

        aco_tsp = ACO_TSP(random_state=100)
        results_aco_tsp = get_analysis(aco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_tsp"].append([results_aco_tsp[key] for key in ["fitness","run_time"]])

        haco_vrp = HACO_VRP(random_state=100)
        results_haco_vrp = get_analysis(haco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_vrp"].append([results_haco_vrp[key] for key in ["fitness","run_time"]])

        haco_tsp = HACO_TSP(random_state=100)
        results_haco_tsp = get_analysis(haco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_tsp"].append([results_haco_tsp[key] for key in ["fitness","run_time"]])
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  10
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  15
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  1

In [42]:
run_time_analysis

{'aco_vrp': [[[0.7161728395061729, 0.21090292930603027],
   [0.7187480215257992, 0.19717168807983398],
   [0.6585390946502064, 0.2010972499847412],
   [0.7009259259259261, 0.25295233726501465],
   [0.71712962962963, 0.22220802307128906],
   [0.7259259259259259, 0.21281766891479492],
   [0.700771604938272, 0.214402437210083],
   [0.7218621399176954, 0.21146583557128906],
   [0.676491769547325, 0.28830409049987793],
   [0.7166152263374485, 0.2587769031524658],
   [0.714146090534979, 0.1959214210510254],
   [0.7387158623269734, 0.19889426231384277],
   [0.7006172839506176, 0.2990233898162842],
   [0.7245576131687242, 0.24901032447814941],
   [0.6774691358024683, 0.26285791397094727],
   [0.7081275720164606, 0.21309614181518555],
   [0.6507716049382725, 0.2545325756072998],
   [0.7127057613168724, 0.26497578620910645],
   [0.7175925925925926, 0.19814610481262207],
   [0.895679012345679, 0.20348429679870605]],
  [[0.6603189300411522, 0.7019989490509033],
   [0.6783950617283949, 0.5408453941

In [43]:
result_to_xlsx(run_time_analysis,n,random_ls,"3_various_numbers_pois_all_1_haco2.xlsx")

## DOI Combination

In [12]:
def result_to_xlsx_doicombi(result,doi_ls,random_ls,filename):
    df_result = pd.DataFrame()
    for method in result:
        for i_doi in range(len(result[method])):
            res_method = result[method][i_doi]
            temp = {
                "random_seed" : random_ls,
                "fitness" : [],
                "fitness_1" : [],
                "fitness_2" : [],
                "fitness_3" : [],
                "poi_included" : [],
                "rating" : [],
                "tarif" : [],
                "total_duration" : [],
                "run_time" : [],
            }
            for res in res_method:
                temp["fitness"].append(res['fitness'])
                temp["fitness_1"].append(res['daily_fitness'][0])
                temp["fitness_2"].append(res['daily_fitness'][1])
                temp["fitness_3"].append(res['daily_fitness'][2])
                temp["poi_included"].append(res['poi_included'])
                temp["rating"].append(res['avg_rating'])
                temp["tarif"].append(res['total_tarif'])
                temp["total_duration"].append(res['total_duration'])
                temp["run_time"].append(res['run_time'])
            temp = pd.DataFrame(temp)
            temp['dwaktu'] = doi_ls[i_doi][0]
            temp['dtarif'] = doi_ls[i_doi][1]
            temp['drating'] = doi_ls[i_doi][2]
            temp['method'] = method
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

In [5]:
doi_ls = [[1,1,1],[1,1,0],[1,0,1],[0,1,1],[1,0,0],[0,0,1],[0,1,0]]
doi_analysis = {
    "aco_vrp": [],
    "aco_tsp": [],
    "haco_vrp": [],
    "haco_tsp": []
}

for i in doi_ls:
    print("DOI = ",i)
    random_ls = list(range(5,(5*50)+1,5))
    random_node = {
        "aco_vrp": [],
        "aco_tsp": [],
        "haco_vrp": [],
        "haco_tsp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = i[0],i[1],i[2]
        
        print("random_state : ",random_ls[j])

        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_vrp"].append(results_aco_vrp)

        aco_tsp = ACO_TSP(random_state=100)
        results_aco_tsp = get_analysis(aco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_tsp"].append(results_aco_tsp)

        haco_vrp = HACO_VRP(random_state=100)
        results_haco_vrp = get_analysis(haco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_vrp"].append(results_haco_vrp)

        haco_tsp = HACO_TSP(random_state=100)
        results_haco_tsp = get_analysis(haco_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["haco_tsp"].append(results_haco_tsp)
        
    for method in doi_analysis:
        doi_analysis[method].append(random_node[method])

DOI =  [1, 1, 1]
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
DOI

In [6]:
doi_analysis

{'aco_vrp': [[{'solution': [[7, 50, 88, 52, 74, 9, 2, 93, 15],
     [64, 10, 22, 14, 35, 4, 66],
     [51, 55, 65, 26]],
    'fitness': 0.6582518052643839,
    'daily_fitness': [0.7192078674586537,
     0.6327714759908156,
     0.6132643256464011],
    'days_needed': 3,
    'poi_included': 20,
    'avg_rating': 4.635,
    'total_tarif': 60000,
    'total_duration': 124560,
    'run_time': 5.757712364196777},
   {'solution': [[46, 89, 74, 2, 44, 71, 49],
     [12, 39, 34, 66, 57, 36, 21],
     [6, 18, 99, 32, 22, 10]],
    'fitness': 0.6434329363778725,
    'daily_fitness': [0.6287432106569275,
     0.61060351992321,
     0.6018057604064242],
    'days_needed': 3,
    'poi_included': 20,
    'avg_rating': 4.575,
    'total_tarif': 27000,
    'total_duration': 119640,
    'run_time': 7.697288513183594},
   {'solution': [[85, 46, 53, 50, 74, 2, 3, 15, 36],
     [8, 20, 64, 33, 57],
     [81, 32, 39, 68, 93, 43]],
    'fitness': 0.595973738960719,
    'daily_fitness': [0.6056657311742497,


In [13]:
result_to_xlsx_doicombi(doi_analysis,doi_ls,random_ls,"doi_combination_haco.xlsx")

## Uji Statistik

In [17]:
def uji_statistik(first_data,second_data,alternative="greater",first_data_name="first",second_data_name="second"):
    # first_data is the target data, e.g. the results of Hybrid ACS-BSO 
    # second_data is a list of results from other method
    # the test is using Wilcoxon Rank-Sum Test
    second_data_name = ["second" for i in range(len(second_data))] if second_data_name == "second" else second_data_name
    for i in range(len(second_data)):
        statistic, p_value = stats.ranksums(first_data, second_data[i],alternative=alternative)
        print(f"Wilcoxon Rank-Sum Test ({first_data_name} vs. {second_data_name[i]}):")
        print("Test Statistic:", statistic)
        print("p-value:", p_value)
        print("")

### Power Analysis

In [105]:
import numpy as np
from scipy.stats import ranksums

# Parameters for power analysis
alpha = 0.05         # Significance level
effect_size = 0.5    # Desired effect size (e.g., Cohen's d)

# Sample sizes for the two groups
n1 = 50
n2 = 50

# Number of simulations
num_simulations = 1000

# Count of rejections
reject_count = 0

# Perform power analysis through simulation
np.random.seed(42)
for _ in range(num_simulations):
    # Simulate data under the alternative hypothesis with the desired effect size
    group1 = np.random.normal(loc=0, scale=1, size=n1)
    group2 = np.random.normal(loc=effect_size, scale=1, size=n2)
    
    # Perform Wilcoxon rank-sum test
    _, p_value = ranksums(group1, group2)
    
    # Check if null hypothesis is rejected
    if p_value < alpha:
        reject_count += 1

# Calculate power
power = reject_count / num_simulations
print("Power:", power)


Power: 0.7


### Results Comparison

In [2]:
results_df1 = pd.read_excel("3_various_numbers_pois_all_1_haco.xlsx")
results_df2 = pd.read_excel("3_various_numbers_pois_all_1_haco2.xlsx")
results_df = pd.concat([results_df1,results_df2])
results_df = results_df.reset_index(drop=True)
results_df.head()

,fitness,run_time,n,method,random_seed
0,0.698302,0.241487,5,aco_vrp,5
1,0.706605,0.240623,5,aco_vrp,10
2,0.701337,0.239109,5,aco_vrp,15
3,0.710905,0.302034,5,aco_vrp,20
4,0.712088,0.212518,5,aco_vrp,25


In [3]:
results_df.groupby(['method','n']).mean()

fitness   run_time  random_seed
method   n                                   
aco_tsp  5   0.700716   0.446448        127.5
         10  0.649839   1.088979        127.5
         15  0.610213   2.104724        127.5
         20  0.603247   3.336888        127.5
         25  0.586110   4.617527        127.5
...               ...        ...          ...
haco_vrp 65  0.577743  11.339346        127.5
         70  0.572882  11.734279        127.5
         75  0.569047  13.493015        127.5
         80  0.562864  14.025830        127.5
         87  0.558138  16.674079        127.5

[68 rows x 3 columns]

In [4]:
results_df.groupby(['method','n']).mean().to_excel('hana_results_comparison.xlsx')

In [53]:
n = results_df['n'].unique()
n.sort()
metric = "fitness"
alternative = "greater"

for i in n:
    print("n: ",i)
    first_data = results_df[(results_df['method']=="haco_vrp")&(results_df['n']==i)][metric].values
    first_data_name = "haco_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "haco_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&(results_df['n']==i)][metric].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

n:  5
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 0.1723454968864278
p-value: 0.4315829590466157

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 2.3887085868458895
p-value: 0.008453853751595578

Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 2.3094296582781326
p-value: 0.010459875418564538

n:  10
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 0.5928685092893117
p-value: 0.2766345771187626

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 2.943661086820187
p-value: 0.0016217748840709728

Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 2.688589751428274
p-value: 0.0035877271741693913

n:  15
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 1.4614898135969079
p-value: 0.07194053450305202

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 7.348811987237283
p-value: 9.998811874512876e-14

Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 6.8317754965779

In [54]:
n = results_df['n'].unique()
n.sort()
metric = "fitness"
alternative = "greater"

first_name = ['haco_vrp','aco_vrp']
second_name = ['haco_tsp','aco_tsp']

for i,j in zip(first_name,second_name):
    print(f"{i} VS {j}")
    for k in n:
        print("n: ",k)
        first_data = results_df[(results_df['method']==i)&(results_df['n']==k)][metric].values
        first_data_name = i

        second_data = [results_df[(results_df['method']==j)&(results_df['n']==k)][metric].values]
        second_data_name = [j]

        uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

haco_vrp VS haco_tsp
n:  5
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 2.3094296582781326
p-value: 0.010459875418564538

n:  10
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 2.688589751428274
p-value: 0.0035877271741693913

n:  15
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 6.831775496577999
p-value: 4.193501740432779e-12

n:  20
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.086450713911193
p-value: 3.0714279138359595e-16

n:  25
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.279477670423992
p-value: 6.185836456340922e-17

n:  30
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.955468136277508
p-value: 8.922772305973768e-16

n:  35
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.762441179764709
p-value: 4.165500671706312e-15

n:  40
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.176070372292136
p-value: 1.466253815362364e-16

n:  45
Wilc

### DOI Combination

In [14]:
results_df = pd.read_excel("doi_combination_haco.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,dwaktu,dtarif,drating,method
0,5,0.658252,0.719208,0.632771,0.613264,20,4.635000,60000,124560,5.757712,1,1,1,aco_vrp
1,10,0.643433,0.628743,0.610604,0.601806,20,4.575000,27000,119640,7.697289,1,1,1,aco_vrp
2,15,0.595974,0.605666,0.555526,0.560205,20,4.650000,101500,124500,5.224181,1,1,1,aco_vrp
3,20,0.663172,0.685587,0.628119,0.594880,21,4.609524,55000,122760,6.305201,1,1,1,aco_vrp
4,25,0.603657,0.636395,0.590555,0.582934,19,4.600000,109000,128700,5.997119,1,1,1,aco_vrp


In [15]:
results_df.groupby(['method','dwaktu','dtarif','drating']).mean().reset_index()

,method,dwaktu,dtarif,drating,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
0,aco_tsp,0,0,1,127.5,0.704900,0.533641,0.393020,0.333994,17.64,4.611572,145080.0,123007.2,29.920908
1,aco_tsp,0,1,0,127.5,0.657602,0.460293,0.528337,0.538819,17.86,4.475283,180410.0,122571.6,33.086453
2,aco_tsp,0,1,1,127.5,0.677868,0.663060,0.534251,0.482247,18.04,4.590358,144190.0,123747.6,69.056649
3,aco_tsp,1,0,0,127.5,0.585441,0.468023,0.436796,0.447995,19.64,4.555013,178830.0,122995.2,21.859793
4,aco_tsp,1,0,1,127.5,0.580727,0.581543,0.470045,0.467784,18.42,4.601014,158750.0,123342.0,18.214255
5,aco_tsp,1,1,0,127.5,0.558919,0.603005,0.549951,0.566110,19.66,4.552205,184250.0,123434.4,17.468180
6,aco_tsp,1,1,1,127.5,0.573938,0.662548,0.558699,0.545581,18.92,4.594106,163300.0,124737.6,18.675905
7,aco_vrp,0,0,1,127.5,0.777948,0.511966,0.442795,0.390451,21.84,4.609700,149970.0,130809.6,8.218627
8,aco_vrp,0,1,0,127.5,0.784372,0.574835,0.571620,0.572722,21.16,4.541634,89640.0,128496.0,8.247129
9,aco_vrp,0,1,1,127.5,0.757139,0.636555,0.591389,0.565876,20.80,4.603334,82240.0,128374.8,6.884783


In [70]:
doi_ls

[[1, 1, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0]]

In [18]:
doi_ls = [[1, 1, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0]]
metric = "fitness"
alternative = "greater"

for i in doi_ls:
    print("doi_combi: ",i)
    first_data = results_df[(results_df['method']=="haco_vrp")&
                            (results_df['dwaktu']==i[0])&
                            (results_df['dtarif']==i[1])&
                            (results_df['drating']==i[2])][metric].values
    first_data_name = "haco_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "haco_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&
                                      (results_df['dwaktu']==i[0])&
                                      (results_df['dtarif']==i[1])&
                                      (results_df['drating']==i[2])][metric].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

doi_combi:  [1, 1, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 2.5989700930473316
p-value: 0.004675195962260762

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 7.7417597201383375
p-value: 4.902507923826636e-15

Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.955468136277508
p-value: 8.922772305973768e-16

doi_combi:  [1, 1, 0]
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 3.0815374843293295
p-value: 0.0010296727304118485

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 8.513867546189534
p-value: 8.411308269177677e-18

Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.59659338469502
p-value: 4.105865323277908e-18

doi_combi:  [1, 0, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. aco_vrp):
Test Statistic: 0.8203645651793964
p-value: 0.20600415478765394

Wilcoxon Rank-Sum Test (haco_vrp vs. aco_tsp):
Test Statistic: 6.287163726416887
p-value: 1.6165917304619092e-10

Wilcoxon Rank-Sum Test (hac

In [19]:
doi_ls = [[1, 1, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0]]
metric = "fitness"
alternative = "greater"

first_name = ['haco_vrp','aco_vrp']
second_name = ['haco_tsp','aco_tsp']

for i,j in zip(first_name,second_name):
    print(f"{i} VS {j}")
    for k in doi_ls:
        print("doi_combi: ",k)
        first_data = results_df[(results_df['method']==i)&
                                (results_df['dwaktu']==k[0])&
                                (results_df['dtarif']==k[1])&
                                (results_df['drating']==k[2])][metric].values
        first_data_name = i

        second_data = [results_df[(results_df['method']==j)&
                                  (results_df['dwaktu']==k[0])&
                                  (results_df['dtarif']==k[1])&
                                  (results_df['drating']==k[2])][metric].values]
        second_data_name = [j]

        uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

haco_vrp VS haco_tsp
doi_combi:  [1, 1, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.955468136277508
p-value: 8.922772305973768e-16

doi_combi:  [1, 1, 0]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.59659338469502
p-value: 4.105865323277908e-18

doi_combi:  [1, 0, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 5.880428353764917
p-value: 2.0460305263638752e-09

doi_combi:  [0, 1, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.748653540013795
p-value: 4.643601386765605e-15

doi_combi:  [1, 0, 0]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 6.483637592867415
p-value: 4.4768573197484855e-11

doi_combi:  [0, 0, 1]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 7.279873788482711
p-value: 1.6706631379618705e-13

doi_combi:  [0, 1, 0]
Wilcoxon Rank-Sum Test (haco_vrp vs. haco_tsp):
Test Statistic: 8.617274844321392
p-value: 3.428320723737822e-18

aco_vrp VS aco_tsp
doi_co